# Create an empty Ubuntu VM on Azure

Install HAGrid

```
$ pip install hagrid
```

Run hagrid launch with these arguments:

```
$ hagrid launch to azure --image_name=domain_0.7.0 --jupyter --ansible_extras="install=false"
```

What ever you enter into Repo and Branch will be ignored

![ip_address](img/hagrid_bare_vm.png)

After it is finished you should see this message

![ip_address](img/hagrid_bare_vm_output.png)

Now provide the `HOST_IP`, Username and Key to the person who is provisioning the system